In [ ]:
from google.transit import gtfs_realtime_pb2
import json
from pathlib import Path
import requests 
from datetime import datetime

In [ ]:
# Read credentials
credentials_path = Path("../config/digitransit_keys.json")
with open(credentials_path, mode="r", encoding="utf-8" ) as credentials_json:
    credentials = json.load(credentials_json)
# Endpoints
service_alerts = f"https://realtime.hsl.fi/realtime/service-alerts/v2/hsl?digitransit-subscription-key={credentials['primary_key']}"
vehicle_positions = f"https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl?digitransit-subscription-key={credentials['primary_key']}"
trip_updates = f"https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl?digitransit-subscription-key={credentials['primary_key']}"

In [ ]:
def save_response(response: requests.models.Response) -> None:
    with open(f'{datetime.now()}', "wb") as file:
        file.write(response.content)

In [ ]:
# vehicle_positions
response = requests.get(vehicle_positions)
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)
for entity in feed.entity:
    print(entity)

In [ ]:
# Change links to services to those made by compose naming conventions
  minio:
    image: minio/minio:latest
    command: server /data --console-address ":9001"
    environment:
      MINIO_ROOT_USER: minioadmin
      MINIO_ROOT_PASSWORD: minioadmin
    ports:
      - "9000:9000"  # S3 API
      - "9001:9001"  # Web UI
    volumes:
      - minio-data:/data
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9000/minio/health/live"]
      interval: 30s
      timeout: 20s
      retries: 3
    restart: always
 minio-mc:
    image: minio/mc:latest
    entrypoint: >
      /bin/sh -c "
      until mc alias set myminio http://minio:9000 minioadmin minioadmin; do
        echo 'Waiting for MinIO...'
        sleep 2
      done
      mc mb myminio/airflow-bucket --ignore-existing
      mc mb myminio/raw-data --ignore-existing
      mc mb myminio/processed-data --ignore-existing
      echo 'MinIO buckets created successfully'
      tail -f /dev/null
      "
    depends_on:
      - minio

  dbt:
    image: fishtownanalytics/dbt:1.7.0
    working_dir: /usr/app
    volumes:
      - ${AIRFLOW_PROJ_DIR:-.}/dbt:/usr/app
      - ${AIRFLOW_PROJ_DIR:-.}/dbt/profiles.yml:/root/.dbt/profiles.yml
    environment:
      DBT_PROFILES_DIR: /root/.dbt
    command: >
      /bin/sh -c "
      until nc -z postgres 5432; do
        echo 'Waiting for PostgreSQL...'
        sleep 2
      done
      tail -f /dev/null
      "

In [1]:
pip install minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.3 MB/s  0:00:003.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [minio]
Note: you may need to restart the kernel to use updated packages.
